In [1]:
%pip install selenium pandas openpyxl

Note: you may need to restart the kernel to use updated packages.


In [2]:
import time
import random
import pandas as pd
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    TimeoutException,
    NoSuchElementException,
    WebDriverException
)

# Setup for headless Chrome and webdriver
def setup_driver():
    options = Options()
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--window-size=1920,1080')
    options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36')
    service = Service('C:\\Users\\yujit\\OneDrive\\Desktop\\chromedriver-win64\\chromedriver.exe')
    driver = webdriver.Chrome(service=service, options=options)
    return driver

# Random waiting function to mimic human browsing behavior
def random_wait():
    wait_time = random.uniform(2, 5)
    print(f"Waiting for {wait_time:.2f} seconds...")
    time.sleep(wait_time)

# Function to scrape YouTube videos
def scrape_youtube_videos(url="https://www.youtube.com/@GenshinImpact/videos"):
    driver = setup_driver()
    driver.get(url)
    random_wait()
    
    video_data = []
    scraped_videos = set()
    try:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "contents")))
        
        while True:
            videos = driver.find_elements(By.XPATH, "//ytd-rich-item-renderer")
            
            for video in videos:
                try:
                    title_element = video.find_element(By.XPATH, ".//yt-formatted-string[@id='video-title']")
                    title = title_element.text
                    if title in scraped_videos:
                        continue
                    views = video.find_element(By.XPATH, ".//span[@class='inline-metadata-item style-scope ytd-video-meta-block']").text
                    upload_date = video.find_elements(By.XPATH, ".//span[@class='inline-metadata-item style-scope ytd-video-meta-block']")[1].text
                    
                    video_data.append([title, views, upload_date])
                    scraped_videos.add(title)
                except NoSuchElementException:
                    continue
            
            last_height = driver.execute_script("return document.documentElement.scrollHeight")
            driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
            random_wait()
            new_height = driver.execute_script("return document.documentElement.scrollHeight")
            
            if new_height == last_height:
                break
    except TimeoutException:
        print("Timed out waiting for page elements to load")
    
    driver.quit()
    
    # Save data to Excel
    df = pd.DataFrame(video_data, columns=["Title", "Views", "Upload Date"])
    df.to_excel("youtube_videos.xlsx", index=False)
    print("Data saved to youtube_videos.xlsx")

# Example Usage
scrape_youtube_videos("https://www.youtube.com/@GenshinImpact/videos")

Waiting for 2.51 seconds...
Waiting for 3.44 seconds...
Waiting for 4.57 seconds...
Waiting for 2.82 seconds...
Waiting for 2.96 seconds...
Waiting for 3.81 seconds...
Waiting for 3.08 seconds...
Waiting for 4.50 seconds...
Waiting for 4.71 seconds...
Waiting for 3.78 seconds...
Waiting for 4.04 seconds...
Waiting for 4.16 seconds...
Waiting for 4.99 seconds...
Waiting for 3.12 seconds...
Waiting for 3.18 seconds...
Waiting for 4.15 seconds...
Waiting for 4.44 seconds...
Waiting for 2.40 seconds...
Waiting for 4.60 seconds...
Waiting for 2.26 seconds...
Waiting for 2.72 seconds...
Waiting for 4.83 seconds...
Data saved to youtube_videos.xlsx
